In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import nibabel as nib

# Set the path to the dataset folder
dataset_path = "Datasets"

# Set the path to the scans and labels folders
scans_path = os.path.join(dataset_path, "Scans")
labels_path = os.path.join(dataset_path, "Labels")

# Get the list of scan and label files
scan_files = sorted(os.listdir(scans_path))
label_files = sorted(os.listdir(labels_path))

# Load the scans and labels into numpy arrays
scans = []
labels = []
for scan_file, label_file in zip(scan_files, label_files):
    scan = nib.load(os.path.join(scans_path, scan_file)).get_fdata()
    label = nib.load(os.path.join(labels_path, label_file)).get_fdata()
    scan /= np.max(scan)  # Normalize the scan image
    label /= np.max(label)  # Normalize the label image
    scans.append(scan)
    labels.append(label)

# Convert the lists to numpy arrays
scans = np.array(scans)
labels = np.array(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(scans, labels, test_size=0.2, random_state=42)

# Define the U-Net model architecture
inputs = layers.Input(shape=(None, None, 1))
conv1 = layers.Conv2D(64, 3, activation="relu", padding="same")(inputs)
conv1 = layers.Conv2D(64, 3, activation="relu", padding="same")(conv1)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
conv2 = layers.Conv2D(128, 3, activation="relu", padding="same")(pool1)
conv2 = layers.Conv2D(128, 3, activation="relu", padding="same")(conv2)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
conv3 = layers.Conv2D(256, 3, activation="relu", padding="same")(pool2)
conv3 = layers.Conv2D(256, 3, activation="relu", padding="same")(conv3)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)
conv4 = layers.Conv2D(512, 3, activation="relu", padding="same")(pool3)
conv4 = layers.Conv2D(512, 3, activation="relu", padding="same")(conv4)
drop4 = layers.Dropout(0.5)(conv4)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)
conv5 = layers.Conv2D(1024, 3, activation="relu", padding="same")(pool4)
conv5 = layers.Conv2D(1024, 3, activation="relu", padding="same")(conv5)
drop5 = layers.Dropout(0.5)(conv5)
up6 = layers.Conv2D(512, 2, activation="relu", padding="same")(layers.UpSampling2D(size=(2, 2))(drop5))
merge6 = layers.concatenate([drop4, up6], axis=3)
conv6 = layers.Conv2D(512, 3, activation="relu", padding="same")(merge6)
conv6 = layers.Conv2D(512, 3, activation="relu", padding="same")(conv6)
up7 = layers.Conv2D(256, 2, activation="relu", padding="same")(layers.UpSampling2D(size=(2, 2))(conv6))
merge7 = layers.concatenate([conv3, up7], axis=3)
conv7 = layers.Conv2D(256, 3, activation="relu", padding="same")(merge7)
conv7 = layers.Conv2D(256, 3, activation="relu", padding="same")(conv7)
up8 = layers.Conv2D(128, 2, activation="relu", padding="same")(layers.UpSampling2D(size=(2, 2))(conv7))
merge8 = layers.concatenate([conv2, up8], axis=3)
conv8 = layers.Conv2D(128, 3, activation="relu", padding="same")(merge8)
conv8 = layers.Conv2D(128, 3, activation="relu", padding="same")(conv8)
up9 = layers.Conv2D(64, 2, activation="relu", padding="same")(layers.UpSampling2D(size=(2, 2))(conv8))
merge9 = layers.concatenate([conv1, up9], axis=3)
conv9 = layers.Conv2D(64, 3, activation="relu", padding="same")(merge9)
conv9 = layers.Conv2D(64, 3, activation="relu", padding="same")(conv9)
conv10 = layers.Conv2D(3, 1, activation="sigmoid")(conv9)

# Create the model
model = tf.keras.Model(inputs=inputs, outputs=conv10)

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(X_train, y_train, batch_size=16, epochs=50, validation_data=(X_test, y_test))

# Plot the model training graph for the total number of iterations
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Training Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Plot the model loss graph for the total number of iterations
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Use the model to predict segmentations on a test image
test_image = X_test[0]
prediction = model.predict(np.expand_dims(test_image, axis=0))

# Print the segmentation output
segmentation_output = np.argmax(prediction, axis=3)
plt.imshow(segmentation_output[0], cmap="gray")
plt.axis("off")
plt.show()

2023-07-11 00:20:27.154763: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
